**First approach uses CLIP and T5 to encode text and image, after that a traditional FFNN is trained**

## Load the dataset


In [1]:
from VQA_Dataset_CLIP import VQA_Dataset, VQA_Dataset_preloaded, VQA_Dataset_Sentences
import clip
import torch
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import random_split

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
model, preprocess = clip.load("ViT-B/32", device=device)
print(preprocess.__class__.__name__)
model.to(torch.float32)
print("clip model loaded")

Using cuda device
Compose
clip model loaded


In [2]:
print(model)

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

### 2.Precalc clip embedding to train faster

In [3]:
dataset_real = VQA_Dataset_preloaded()
#dataset_real.compute_store(preprocess, model, device, "dataset_real_75k", name="val", length=75000, mode="scale", real=True)

In [4]:
dataset_abstract_train = VQA_Dataset_preloaded()
#dataset_abstract_train.compute_store(preprocess, model, device, "dataset_abstract_train_60k", name="train", length=60000, mode="scale", real=False)

dataset_abstract_test = VQA_Dataset_preloaded()
#dataset_abstract_test.compute_store(preprocess, model, device, "dataset_abstract_test_15k", name="val", length=15000, mode="scale", real=False)

### 3.Only load

In [5]:
from torch.utils.data import ConcatDataset
sentences=False
adding=""
if sentences:
    adding="_sentences_"

dataset_real.load("dataset_real_75k"+adding, device, name="val", length=75000)
dataset_abstract_train.load("dataset_abstract_train_60k"+adding, device, name="train", length=60000)
dataset_abstract_test.load("dataset_abstract_test_15k"+adding, device, name="val", length=15000)
dataset_abstract = ConcatDataset([dataset_abstract_train, dataset_abstract_test])

In [6]:
#Combine abstract and real datasets
class CustomDataset(Dataset):
    def __init__(self, dataset, real=1):
        self.existing_dataset = dataset
        self.real = real

    def __len__(self):
        return len(self.existing_dataset)

    def __getitem__(self, index):
        return  self.existing_dataset[index]+(self.real, )
    
dataset_real_extended = CustomDataset(dataset_real, real=1)
dataset_abstract_extended = CustomDataset(dataset_abstract, real=0)

In [7]:
dataset_type = "combined"
if dataset_type == "real":
    dataset = dataset_real
elif dataset_type == "abstract":
    dataset = dataset_abstract
elif dataset_type == "combined":
    dataset = dataset = ConcatDataset([dataset_real_extended, dataset_abstract_extended])

### Create Dataloader

## New with both Validation and Training Datasets from MSCOCO

In [8]:
batch_size=64

train_size = int(len(dataset)*0.8*0.8)
val_size = int(len(dataset)*0.8*0.2)
test_size = int(len(dataset))-val_size-train_size
print("Train size: ", train_size)
print("Val size: ", val_size)
print("Test size: ", test_size)
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size], generator=generator)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

Train size:  96000
Val size:  24000
Test size:  30000


## Simple model architectue


In [9]:
from models import VQA_Model_Precalc_Zero, VQA_Model1_Precalc, VQA_Model_Precalc

## Evaluate

In [10]:
import numpy as np
class performanceAnalysis:
    def __init__(self):
        self.counters = [0, 0, 0]
        self.total = [0, 0, 0]
    def it(self, correct, answer_types):#both numpy arrays
        for i in range(len(self.counters)):
            self.counters[i]+=np.sum(np.logical_and(correct, answer_types==i))
            self.total[i]+=np.sum(answer_types==i)
    def get_accuracies(self):
        dictionary = {"yes/no": 100*self.counters[0]/self.total[0], "number": 100*self.counters[1]/self.total[1], "other": 100*self.counters[2]/self.total[2], "total": 100*sum(self.counters)/sum(self.total)}
        return dictionary

In [11]:
class performanceAnalysis:
    def __init__(self, dataset_type="combined"):
        self.combined = dataset_type=="combined"
        self.dataset_type = dataset_type
        self.counters = np.zeros((self.combined+1, 3)) 
        self.total = np.zeros((self.combined+1, 3))
    def it(self, correct, answer_types, real_types=None):#both numpy arrays
        if not self.combined:
            for i in range(len(self.counters[0])):
                self.counters[0][i]+=np.sum(np.logical_and(correct, answer_types==i))
                self.total[0][i]+=np.sum(answer_types==i)
        else:
            is_real = (real_types==1).reshape(len(real_types), -1)
            is_abstract = np.logical_not(is_real)
            for i in range(len(self.counters[0])):
                self.counters[0][i]+=np.sum(np.logical_and(np.logical_and(correct, answer_types==i), is_abstract))
                self.total[0][i]+=np.sum(np.logical_and(answer_types==i, is_abstract))

                self.counters[1][i]+=np.sum(np.logical_and(np.logical_and(correct, answer_types==i), is_real))
                self.total[1][i]+=np.sum(np.logical_and(answer_types==i, is_real))

    def get_accuracies(self):
        if not self.combined:
            dictionary = {self.dataset_type: {"yes/no": 100*self.counters[0][0]/self.total[0][0], "number": 100*self.counters[0][1]/self.total[0][1], "other": 100*self.counters[0][2]/self.total[0][2], "total": 100*sum(self.counters[0])/sum(self.total[0])}}
        else:
            dictionary_abstract = {"yes/no": 100*self.counters[0][0]/self.total[0][0], "number": 100*self.counters[0][1]/self.total[0][1], "other": 100*self.counters[0][2]/self.total[0][2], "total": 100*sum(self.counters[0])/sum(self.total[0])}
            dictionary_real = {"yes/no": 100*self.counters[1][0]/self.total[1][0], "number": 100*self.counters[1][1]/self.total[1][1], "other": 100*self.counters[1][2]/self.total[1][2], "total": 100*sum(self.counters[1])/sum(self.total[1])}
            dictionary = {"abstract": dictionary_abstract, "real": dictionary_real} 
        return dictionary
    
def printResults(dict_performance, performAnalysis):
    info = ""
    for i, element in enumerate(dict_performance):
        info += (element+" - ")
        info += f"total: {dict_performance[element]['total']:.2f}, yes/no: {dict_performance[element]['yes/no']:.2f}({performAnalysis.counters[i][0]}/{performAnalysis.total[i][0]}), number: {dict_performance[element]['number']:.2f}({performAnalysis.counters[i][1]}/{performAnalysis.total[i][1]}), other: {dict_performance[element]['other']:.2f}({performAnalysis.counters[i][2]}/{performAnalysis.total[i][2]})"
        if dataset_type=="combined" and i==0:
            info += "\n"
    print(info)

In [12]:
import tqdm

def evaluate(model, dataloader, device, show_progress=False, score_each=False):
    model.eval()
    performAnalysis = performanceAnalysis(dataset_type)
    correct = 0
    if show_progress:
        pbar = tqdm.tqdm(dataloader)
    else:
        pbar = dataloader

    for i, data in enumerate(pbar):
        image = data[0].to(device)
        answer_tokens = data[1].squeeze(0).to(device)
        if dataloader.batch_size == 1:
            correct_answer = torch.tensor([int(data[3])]).to(device)
            question_tokens = data[2].squeeze(0).to(device)
        else:
            correct_answer = torch.tensor([int(x) for x in data[3]]).to(device)
            question_tokens = data[2].squeeze(1).to(device)

        with torch.no_grad():
            if score_each:
                # get similarity for each answer, not for each pair of batch
                if len(answer_tokens.shape) == 2:
                    answer_tokens = answer_tokens.unsqueeze(0)
                similarity = torch.zeros((answer_tokens.shape[0], answer_tokens.shape[1])).to(device)
                
                for answer in range(answer_tokens.shape[1]):    
                    similarity[:,answer] = model(image, question_tokens, answer_tokens[:, answer]).squeeze(1)
                    
                pred = similarity.argmax(dim=-1)
                # get sum of correct answers
                correct += (pred == correct_answer).sum().item()
            else:
                similarity = model(image, question_tokens, answer_tokens)
                pred = similarity.argmax(dim=-1)
                # get sum of correct answers
                correct += (pred == correct_answer).sum().item()
                if dataset_type != "combined":
                    performAnalysis.it((pred==correct_answer).cpu().numpy().reshape(len(pred), -1), data[4].cpu().numpy())
                else:
                    performAnalysis.it((pred==correct_answer).cpu().numpy().reshape(len(pred), -1), data[4].cpu().numpy(), data[5].cpu().numpy())
    dict_performance = performAnalysis.get_accuracies()
    printResults(dict_performance, performAnalysis)
    return dict_performance

In [14]:
combined_model = VQA_Model_Precalc_Zero(model, device)
evaluate(combined_model, test_dataloader, device, show_progress=True);

100%|██████████| 469/469 [00:13<00:00, 35.33it/s]

abstract - total: 18.63, yes/no: 41.79(2560.0/6126.0), number: 5.19(117.0/2253.0), other: 2.13(144.0/6764.0)
real - total: 15.26, yes/no: 35.75(1999.0/5591.0), number: 3.40(63.0/1852.0), other: 2.77(205.0/7414.0)


## Training

In [14]:

from torch.optim import lr_scheduler

def train(model, train_dataloader, val_dataloader, device, epochs=10, patience=3, precalculated=False, save_name="model", loss_fn = torch.nn.CrossEntropyLoss()):
    print(device)
    print(model.parameters())
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)#, weight_decay=1e-4)

    prev_acc = 0.0
    best_acc = 0.0
    count = 0
    
    for epoch in range(epochs):
        inner_bar = tqdm.tqdm(train_dataloader, desc='Batch', colour='green')
        loss_accum = 0
        
        model.train()
        for data in inner_bar:
            
            image = data[0].to(device)
            answer_tokens = data[1].squeeze(0).to(device)
            
            correct_answer = torch.tensor([int(x) for x in data[3]]).to(device)
            question_tokens = data[2].squeeze(1).to(device)
        
            similarity = model(image, question_tokens, answer_tokens)
            loss = loss_fn(similarity, correct_answer)
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=4.0, error_if_nonfinite=True)
            optimizer.step()

            loss_accum += loss.item()
        
        if dataset_type=="combined":
            dic = evaluate(model, val_dataloader, device)
            acc = (dic["abstract"]["total"]+dic["real"]["total"])/2
        else:
            acc = evaluate(model, val_dataloader, device)[dataset_type]["total"]

        if acc > best_acc:
            best_acc = acc
            # save model
            model.save(save_name)
        if acc < prev_acc:
            count += 1
        else:
            count = 0
        prev_acc = acc
        print(f"Epoch {epoch} train_loss: {loss_accum/len(train_dataloader.dataset)}, patience: {count}")
        if count == patience:
            print("early stopping")
            break 
        inner_bar.close()

In [15]:
trained_model = VQA_Model_Precalc(model, device)
# freeze the clip model
for param in trained_model.model.parameters():
    param.requires_grad = False

#evaluate(trained_model, test_dataloader, device, test_size, show_progress=True)
improvement="_dropout2e-1"
filename_save = trained_model.__class__.__name__+"_"+dataset_type+improvement
train(trained_model, train_dataloader, val_dataloader, device, epochs=50, save_name=filename_save)
# evaluate the model
evaluate(trained_model, test_dataloader, device);

cuda
<generator object Module.parameters at 0x0000019AD7CA3D60>


Batch: 100%|██████████| 1500/1500 [00:40<00:00, 36.98it/s]


abstract - total: 43.09, yes/no: 50.87(2490.0/4895.0), number: 39.55(664.0/1679.0), other: 37.12(1997.0/5380.0)
real - total: 37.92, yes/no: 53.35(2407.0/4512.0), number: 25.31(387.0/1529.0), other: 29.54(1774.0/6005.0)
Epoch 0 train_loss: 0.03169110848878821, patience: 0


Batch: 100%|██████████| 1500/1500 [00:40<00:00, 37.14it/s]


abstract - total: 46.08, yes/no: 53.42(2615.0/4895.0), number: 40.38(678.0/1679.0), other: 41.19(2216.0/5380.0)
real - total: 39.55, yes/no: 50.55(2281.0/4512.0), number: 30.80(471.0/1529.0), other: 33.51(2012.0/6005.0)
Epoch 1 train_loss: 0.0261961359915634, patience: 0


...

abstract - total: 54.04, yes/no: 64.74(3169.0/4895.0), number: 40.50(680.0/1679.0), other: 48.53(2611.0/5380.0)
real - total: 45.30, yes/no: 60.46(2728.0/4512.0), number: 28.71(439.0/1529.0), other: 38.13(2290.0/6005.0)
Epoch 47 train_loss: 0.019645010529085995, patience: 0


Batch: 100%|██████████| 1500/1500 [00:38<00:00, 38.50it/s]


abstract - total: 55.06, yes/no: 63.80(3123.0/4895.0), number: 44.13(741.0/1679.0), other: 50.52(2718.0/5380.0)
real - total: 46.16, yes/no: 59.49(2684.0/4512.0), number: 30.09(460.0/1529.0), other: 40.25(2417.0/6005.0)
Epoch 48 train_loss: 0.01946604947807888, patience: 0


Batch: 100%|██████████| 1500/1500 [00:39<00:00, 38.43it/s]


abstract - total: 53.66, yes/no: 59.80(2927.0/4895.0), number: 45.98(772.0/1679.0), other: 50.46(2715.0/5380.0)
real - total: 45.07, yes/no: 57.03(2573.0/4512.0), number: 30.35(464.0/1529.0), other: 39.83(2392.0/6005.0)
Epoch 49 train_loss: 0.019484972908472023, patience: 1
abstract - total: 52.88, yes/no: 59.39(3638.0/6126.0), number: 44.25(997.0/2253.0), other: 49.85(3372.0/6764.0)
real - total: 44.80, yes/no: 56.36(3151.0/5591.0), number: 29.59(548.0/1852.0), other: 39.88(2957.0/7414.0)


In [16]:
trained_model = VQA_Model1_Precalc(model, device)
# freeze the clip model
for param in trained_model.model.parameters():
    param.requires_grad = False

#evaluate(trained_model, test_dataloader, device, test_size, show_progress=True)
filename_save = trained_model.__class__.__name__+"_"+dataset_type
train(trained_model, train_dataloader, val_dataloader, device, epochs=50, save_name=filename_save)
# evaluate the model
evaluate(trained_model, test_dataloader, device);

cuda
<generator object Module.parameters at 0x00000237D7CE1CF0>


Batch: 100%|██████████| 1500/1500 [00:42<00:00, 35.09it/s]


abstract - total: 48.98, yes/no: 54.71(2678.0/4895.0), number: 33.11(556.0/1679.0), other: 48.72(2621.0/5380.0)
real - total: 44.05, yes/no: 46.10(2080.0/4512.0), number: 25.18(385.0/1529.0), other: 47.31(2841.0/6005.0)
Epoch 0 train_loss: 0.12611973020186026, patience: 0


Batch: 100%|██████████| 1500/1500 [00:43<00:00, 34.88it/s]


abstract - total: 51.50, yes/no: 55.02(2693.0/4895.0), number: 39.55(664.0/1679.0), other: 52.03(2799.0/5380.0)
real - total: 45.83, yes/no: 47.94(2163.0/4512.0), number: 25.83(395.0/1529.0), other: 49.34(2963.0/6005.0)
Epoch 1 train_loss: 0.0976900674564143, patience: 0


...

abstract - total: 54.00, yes/no: 59.51(2913.0/4895.0), number: 30.49(512.0/1679.0), other: 56.32(3030.0/5380.0)
real - total: 49.48, yes/no: 56.07(2530.0/4512.0), number: 28.32(433.0/1529.0), other: 49.91(2997.0/6005.0)
Epoch 10 train_loss: 0.06319272099683682, patience: 1


Batch: 100%|██████████| 1500/1500 [00:43<00:00, 34.49it/s]


abstract - total: 55.49, yes/no: 58.20(2849.0/4895.0), number: 37.22(625.0/1679.0), other: 58.72(3159.0/5380.0)
real - total: 47.58, yes/no: 51.53(2325.0/4512.0), number: 25.77(394.0/1529.0), other: 50.16(3012.0/6005.0)
Epoch 11 train_loss: 0.0626291747558862, patience: 2


Batch: 100%|██████████| 1500/1500 [00:43<00:00, 34.32it/s]


abstract - total: 52.93, yes/no: 55.53(2718.0/4895.0), number: 28.65(481.0/1679.0), other: 58.14(3128.0/5380.0)
real - total: 47.48, yes/no: 50.64(2285.0/4512.0), number: 27.60(422.0/1529.0), other: 50.17(3013.0/6005.0)
Epoch 12 train_loss: 0.061971628256142136, patience: 3
early stopping
abstract - total: 53.29, yes/no: 55.24(3384.0/6126.0), number: 30.14(679.0/2253.0), other: 59.24(4007.0/6764.0)
real - total: 47.22, yes/no: 49.44(2764.0/5591.0), number: 26.62(493.0/1852.0), other: 50.69(3758.0/7414.0)


## Save the model!

In [ ]:
# save the model
trained_model.save("trained_model_")

## Load and evaluate


In [17]:
trained_model = VQA_Model_Precalc(model, device) # need to choose the same model!
dataset_type="combined"
improvement="_dropout2e-1"
filename_load = trained_model.__class__.__name__+"_"+dataset_type+improvement
trained_model.load(filename_load)
#print(trained_model)
evaluate(trained_model, test_dataloader, device, show_progress=True);

100%|██████████| 469/469 [00:12<00:00, 36.59it/s]

abstract - total: 54.31, yes/no: 62.00(3798.0/6126.0), number: 43.41(978.0/2253.0), other: 50.98(3448.0/6764.0)
real - total: 46.01, yes/no: 59.79(3343.0/5591.0), number: 29.54(547.0/1852.0), other: 39.74(2946.0/7414.0)
